# How can we make custom http calls and get JSON using Graph SDK v5 (IRequestAdapter)

Below code is not executable by its own. It require methods to get the `GraphServiceClient` and the `IConfigurationRoot`

In [ ]:
class MyDrive
{
    [JsonPropertyName("name")]
    public string Name { get; set; }
    [JsonPropertyName("id")]
    public string Id { get; set; }
}

In [ ]:
//Arrange
GraphServiceClient graphServiceClient = GraphServiceClientFactory.GetGraphServiceClientAsync();
IConfigurationRoot config = ConfigProvider.GetConfigRoot();
string driveId = config["DriveId"];
RequestInformation driveReqInfo = new() { URI = new Uri($"https://graph.microsoft.com/v1.0/drives/{driveId}")};

IRequestAdapter adp = graphServiceClient.RequestAdapter;
var nativeHandler = new NativeResponseHandler();
driveReqInfo.SetResponseHandler(nativeHandler);

//Act
// Giving a dummy object to deserialize
var baseItem = await adp.SendAsync<BaseItem>(driveReqInfo, (parsable) => null);
var response = nativeHandler.Value as HttpResponseMessage;
string rawJSONResponse = await response?.Content?.ReadAsStringAsync();
MyDrive myDrive= JsonSerializer.Deserialize<MyDrive>(rawJSONResponse);

//Assert
Assert.IsInstanceOfType(response, typeof(HttpResponseMessage));
Assert.IsNotNull(rawJSONResponse);
Assert.AreEqual(66, myDrive.Id.Length, "Length of Graph Id of drive is not 66");